## Import packages

In [1]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd
import numpy as np
import httpx
from datetime import datetime

## Enter item key-word to scrape

In [31]:
driver_path = r"C:\Users\MACHENIKE\Downloads\Installer\chromedriver_win32\chromedriver.exe"

all_urls = []
output = pd.DataFrame(columns=['Index','Shop_Name', 'Prod_Name', 'Price', 'Shipping_From','Unit_Sold', 'Average_Rating', 'Total_Comments',
                               'Buyer_Rating', 'Buyer_Review', 'Buyer_Tags','Review_Time', 'URL'])

keyword_input=input("search keyword: ")
#url_input = input('Enter your scrape target url:')
file_name = input('Enter your file name:')

if ' ' in keyword_input:
     keyword = keyword_input.replace(' ' , '%20')
else:
     keyword = keyword_input

url_input = 'https://shopee.com.my/search?keyword=' + str(keyword)
#url_input="https://shopee.com.my/api/v2/search_items/?by=relevancy&keyword="+name+"&limit=50&newest=0&order=desc&page_type=search&version=2"


search keyword: smartphone
Enter your file name:smartphone


## Scrape listings from Shopee

In [32]:
start_time = time.time()

driver = webdriver.Chrome(executable_path = driver_path)

for page in range(1):
    
#    product_list = 'https://shopee.com.my/search?keyword=' + str(keyword) + '&page=' + str(page)
    driver.get(url_input)

    if page == 0:
        WebDriverWait(driver,5).until(EC.element_to_be_clickable((By.XPATH,'//div[@class="language-selection__list-item"]/button'))).click()
    else:
        WebDriverWait(driver,5)
    time.sleep(3)

    for x in range(10):
        driver.execute_script("window.scrollBy(0,300)")
        time.sleep(0.1)

    all_items = driver.find_elements_by_xpath('//a[@data-sqe="link"]')

    for item in all_items:
        url = item.get_attribute('href')
        all_urls.append(url)

driver.quit()
        
index = 1

#------------------------------------------------------------------------------------------------------------------------#
###### edit your selection to increase total number of shops scrape ######

prod_url=all_urls[0:30] ##### scrape the first 5 listings - 1st row in shopee webpage ######
# prod_url=all_urls[5:10] ##### scrape the second 5 listings - 2nd row in shopee webpage ######
#prod_url=all_urls[10:15] ##### scrape the second 5 listings - 2nd row in shopee webpage ######
#prod_url=all_urls[15:20] ##### scrape the second 5 listings - 2nd row in shopee webpage ######

### you may edit the length of this list range if you get a 'timeout error'
#------------------------------------------------------------------------------------------------------------------------#

for url in prod_url:
    ids = url.split('-i.')[1].split('.')
    shop_id = ids[0]
    prod_id = ids[1]
    
    shop_api = 'https://shopee.com.my/api/v2/shop/get?is_brief=1&shopid=' + str(shop_id)
    shop_name = httpx.get(shop_api).json().get('data')['name']

    item_api = 'https://shopee.com.my/api/v2/item/get?itemid=' +str(prod_id) + '&shopid=' + str(shop_id)
    item_detail = httpx.get(item_api).json().get('item')
    price = item_detail['price_min']/100000
    #currency = item_detail['currency']
    shop_location=item_detail['shop_location']
    unit_sold = item_detail['historical_sold']
    prod_name = item_detail['name']
    average_rating = item_detail['item_rating']['rating_star']
    total_rating=item_detail['item_rating']['rcount_with_context']
    
    
    review_api = "https://shopee.com.my/api/v2/item/get_ratings?filter=0&flag=1&itemid="+str(prod_id)+"&limit=6&offset=0&shopid="+str(shop_id)+"&type=0"
    review_detail = httpx.get(review_api).json().get('data')['ratings']
    
    print("t.comments:",total_rating)
    
    list_review_api=[]
    if total_rating <= 100:
        scrape_limit=total_rating
    else:
        scrape_limit=100
        
    count=0
    while count<=scrape_limit:
        time.sleep(1)
        new_review_api="https://shopee.com.my/api/v2/item/get_ratings?filter=0&flag=1&itemid="+str(prod_id)+"&limit=6&offset="+str(count)+"&shopid="+str(shop_id)+"&type=0"
        list_review_api.append(new_review_api)
        count +=6
        #print(list_review_api)
        
    for updated_review_api in list_review_api:
        review_detail = httpx.get(updated_review_api).json().get('data')['ratings']
        if review_detail == []:
                buyer_rating = 'NA'
                buyer_review = 'NA'
                buyer_tags = 'NA'
                buyer_name = 'NA'
                review_time = 'NA'

                output = output.append([{'Index':index, 'Shop_Name':shop_name, 'Prod_Name':prod_name,'Price':price,'Shipping_From':shop_location, 
                                         'Unit_Sold':unit_sold, 'Average_Rating':average_rating, 'Total_Comments':total_rating,
                                         'Buyer_Rating':buyer_rating, 'Buyer_Review':buyer_review,'Buyer_Tags':buyer_tags, 
                                         'Review_Time':review_time, 'URL':url}],sort=False)

        else:
            for i in range(len(review_detail)):
                buyer_rating = review_detail[i]['rating_star']
                buyer_review = review_detail[i]['comment']
                buyer_tags = review_detail[i]['tags']
                buyer_name = review_detail[i]['author_username']
                review_time = datetime.fromtimestamp(review_detail[i]['ctime']).strftime('%Y-%m-%d %H:%M:%S')
                #print(buyer_name)
                output = output.append([{'Index':index, 'Shop_Name':shop_name, 'Prod_Name':prod_name,'Price':price,'Shipping_From':shop_location,
                                         'Unit_Sold':unit_sold, 'Average_Rating':average_rating, 'Total_Comments':total_rating,
                                         'Buyer_Rating':buyer_rating, 'Buyer_Review':buyer_review,'Buyer_Tags':buyer_tags, 
                                         'Review_Time':review_time, 'URL':url}],sort=False)

    index += 1

    print(url)
    #print(len(list_review_api))

output = output.set_index(['Index'])
output.to_csv('ShopeeShoes_' + str(file_name) + '_' + str(datetime.now().strftime('%Y%m%d_%H%M%S')) + '.csv')
        
print("--- %s seconds ---" % (time.time() - start_time))

t.comments: 1600
https://shopee.com.my/realme-6-smartphone-8GB-128GB-64MP-90Hz-Smooth-Display-30W-Flash-Charge-1-Year-Warranty-i.96670954.3439149966
t.comments: 596
https://shopee.com.my/realme-6-Pro-Smartphone-8GB-128GB-6-Cameras-with-64MP-90Hz-Smooth-Display-30W-Flash-Charge-1-year-warranty-i.96670954.3633583083
t.comments: 51
https://shopee.com.my/realme-C15-Smartphone-4GB-64GB-6000-mAh-18W-Quick-Charge-13MP-AI-quad-camera-1-Year-Malaysia-Warranty-i.96670954.5263259813
t.comments: 2
https://shopee.com.my/-TURBO-SHIPPING-OPPO-A15-128GB-ROM-6GB-RAM-With-Facelock-Function-5.5-IPS-LCD-SCREEN-*-New-Import-Smartphone-*-i.353556355.6068644909
t.comments: 4
https://shopee.com.my/Huawei-P40-8GB-RAM-256ROM-5.5-inch-Full-Screen-Smart-Phone-Android-9.1-smartphone-handphone-telefon-i.342200718.3364477304
t.comments: 18
https://shopee.com.my/VIVO-V15-2020-8GB-128GB-ANDROID-5.5INCH-SMART-PHONE-WHATAPPS-YOUTUBE-FACEBOOK-i.328930690.3168992843
t.comments: 6
https://shopee.com.my/I12promax-mobile-pho